# 笔趣阁自动化

In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os

# 抽象爬虫层, 用于识别各种笔趣阁
def gen_metadata():
    return {
        'title': '未知小说',
        'meta': '未找到元数据',
        '作者': '未知作者',
        'chapters': [],
    } # 以上元数据是必须的, chapter格式如下： ('相对路径', '章节标题') 其中章节标题会与网页标题相比较


class Get_2biqu:
    """www.2biqu.com"""
    @staticmethod
    def get_metadata(soup: BeautifulSoup):
        metadata = gen_metadata()

        for c in soup.find('div', class_='top').children:
            if c.text == '\n':
                continue
            if c.name == 'h1':
                metadata['title'] = c.text.strip()
            elif c.name == 'div':
                metadata['meta'] = c.text.strip()

        for c in soup.find('ul', id='section-list').children:
            section = c.text.strip()
            if section == '':
                continue
            if c.name == 'li':
                metadata['chapters'].append((c.a['href'], c.a.text.strip()))
                
        meta_list = metadata['meta'].split('\n')
        meta_list = [i.strip().split('：') for i in meta_list if i.strip() != '']
        for i in meta_list:
            if len(i) == 2:
                metadata[i[0]] = i[1]
            elif len(i) == 1:
                i = i[0].split(':')
                metadata[i[0]] = i[1]
            else:
                print(i)

        return metadata

    @staticmethod
    def get_title(soup: BeautifulSoup):
        return soup.find('h1').text.strip()

    @staticmethod
    def get_content(soup: BeautifulSoup):
        return soup.find('div', id='content').text.strip()
    
    @staticmethod
    def get_content_cut(content: str):
        return [i.strip() for i in content.split('\n') if i.strip() != '']

class Get_52bqg:
    """www.52bqg.com"""
    @staticmethod
    def get_metadata(soup: BeautifulSoup):
        metadata = gen_metadata()
        metadata['meta'] = ""
        for c in soup.find('div', id='info').children:
            if c.text == '\n':
                continue
            if c.name == 'h1':
                metadata['title'] = c.text.strip()
            elif c.name == 'p':
                metadata['meta'] += c.text.strip()+'\n'
                key,value = c.text.strip().split('：')
                metadata[key] = value

        intro = soup.find('div', id='intro').text.strip()
        metadata['meta'] += intro+'\n'

        section_tmp = []
        for c in soup.find_all('dd'):
            if c.text.strip() == '':
                continue
            section_tmp.append((c.a['href'], c.a.text.strip()))
            
        section_tmp = list(reversed(section_tmp))
        seen = set()
        section_tmp = [x for x in section_tmp if not (x in seen or seen.add(x))]
        metadata['chapters'].extend(list(reversed(section_tmp)))
            
        return metadata

    @staticmethod
    def get_title(soup: BeautifulSoup):
        return soup.find('h1').text.strip()

    @staticmethod
    def get_content(soup: BeautifulSoup):
        title = Get_52bqg.get_title(soup) + '\n'
        return title + soup.find('div', id='content').text.strip().replace('\xa0'*4, '\n')
    
    @staticmethod
    def get_content_cut(content: str):
        return [i.strip() for i in content.split('\n') if i.strip() != '']
    
def get_parser(url: str):
    if 'www.2biqu.com' in url:
        print('使用2biqu解析器')
        return Get_2biqu
    if 'www.52bqg.org' in url:
        print('使用52bqg解析器')
        return Get_52bqg
    else:
        raise Exception('未知网站')



前往目标页面

In [8]:
options = webdriver.ChromeOptions()
options.add_argument("disable-blink-features=AutomationControlled")
browser = webdriver.Chrome(options=options)

# root_url = 'https://www.2biqu.com/biqu*****/'
root_url = 'https://www.52bqg.org/book_*****/'

parser = get_parser(root_url)
browser.get(root_url) # 如果有各种问题(如502)请刷新浏览器

使用52bqg解析器


打开一个文本文档用于存放书的内容

In [9]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

metadata = parser.get_metadata(soup)
print("标题：《", metadata['title'], "》", sep='')
print(metadata['meta'])
print("总共有 {} 章节".format(len(metadata['chapters'])))

file_name = metadata['title'] + '.txt' if '作者' not in metadata else metadata['title'] + ' - ' + metadata['作者'] + '.txt'
if os.path.exists(file_name):
    print('文件已存在，将追加内容')
else:
    print('文件不存在，写入元数据')
    with open(file_name, 'w+', encoding='utf-8') as fp:
        fp.write(metadata['title'] + '\n\n')
        fp.write(metadata['meta'] + ' \n\n')
txtfp = open(file_name, 'a', encoding='utf-8')

标题：《神印王座II皓月当空》
作者：唐家三少
动作：投票推荐， 加入书架， 直达底部
更新时间：2024-01-09 10:03 [共326万字]
最新：正文卷 第三百七十九章 凌梦露的底牌
《神印王座》第二部！   龙生两子亦有不同，天才的哥哥与废柴的弟弟，明明是长得一模一样的双胞胎，却有着天壤之别，唯有他们那想要躺平的心却是一模一样。在他们出生的那一晚，皓月当空。

总共有 377 章节
文件不存在，写入元数据


爬取每一章的数据

In [12]:
# 爬虫参数
stop_worlds = ['www.2biqu.com', '笔趣阁', 'www.52bqg.org', '，最快更新神印王座II皓月当空 ！']  # 替换词表, 用于替换内容中的词
end_words = []  # 结束词表，如果没有遇到所有的这些词，就进入没爬完整处理程序
min_content_len = 10  # 最小段落长度
skip = 0 # 跳过多少章节, 0 表示不跳过, 在下一块执行时该值会自动更新


def content_include_all(content, words=end_words):
    # 内容必须包含所有的词
    for i in words:
        if i not in content:
            return False
    return True


def content_include_any(content, words=end_words):
    # 内容仅需要包含一个词
    for i in words:
        if i in content:
            return True
    return False

def content_replace(content, words=stop_worlds):
    # 替换内容中的词
    for i in words:
        content = content.replace(i, '')
    return content

In [13]:
if skip >= len(metadata['chapters']):
    print(f'[ERROR] skip={skip} 大于等于章节数')
    raise Exception('skip 大于等于章节数')
    
if skip > 0:
    print(f'skip={skip} 将从第{skip+1}章开始爬取')

for index, c in enumerate(metadata['chapters']):
    if index < skip:
        continue
    print(f'正在爬取第{index+1}章：{c[1]}')
    browser.get(root_url + c[0])

    # 处理防火墙,重定向与弱网的问题
    title_include = c[1] in browser.title
    while not title_include:
        print('    [WARN] 网页标题与章节标题不符，正在刷新...')
        browser.get(root_url + c[0])
        title_include = c[1] in browser.title
        if title_include:
            print('    网页标题与章节标题符合，开始爬取...')
            break

    # 爬取内容
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    title = parser.get_title(soup)
    content = parser.get_content(soup)
    content = content_replace(content)
    content_cut = parser.get_content_cut(content)
    print(f"    页面标题：{title}, 内容长度：{len(content)}, 段落数：{len(content_cut)}")

    # 爬取内容过少或是有可能没爬完整处理程序
    if len(content_cut) < min_content_len or not content_include_all(content):
        print(f'    [WARN] 段落数小于设定值({min_content_len})或是不包含停止词，可能爬取失败')
        print(f'    [WARN] 输出所有段落：')
        for i in content_cut:
            print('    '+str(i))
        print(f'    [WARN] 输出所有段落结束，输入y/yes/skip/continue保存内容并继续爬取，输入任意内容暂停爬取')
        if input().strip() in ['skip', 'continue', 'y', 'yes']:
            continue
        else:
            print(f'[WARN] 已暂停爬取，当前进度：skip={skip}，index={index}')
            break

    # 将内容写入文件
    txtfp.write('---------- \n\n')
    for i in content_cut:
        txtfp.write(i + '\n\n')
    skip = index
    print(f'    当前章爬取完成')

正在爬取第1章：引子：皓月当空
    页面标题：引子：皓月当空, 内容长度：1572, 段落数：48
    当前章爬取完成
正在爬取第2章：第一章 龙当当与龙空空
    页面标题：第一章 龙当当与龙空空, 内容长度：2622, 段落数：63
    当前章爬取完成
正在爬取第3章：第二章 魔法圣殿
    页面标题：第二章 魔法圣殿, 内容长度：4091, 段落数：83
    当前章爬取完成
正在爬取第4章：第三章 光明庇护体质
    页面标题：第三章 光明庇护体质, 内容长度：4257, 段落数：70
    当前章爬取完成
正在爬取第5章：第四章 龙家的“两大天才”
    页面标题：第四章 龙家的“两大天才”, 内容长度：4466, 段落数：87
    当前章爬取完成
正在爬取第6章：第五章 第一堂课
    页面标题：第五章 第一堂课, 内容长度：3928, 段落数：72
    当前章爬取完成
正在爬取第7章：第六章 圣殿大礼包
    页面标题：第六章 圣殿大礼包, 内容长度：3980, 段落数：76
    当前章爬取完成
正在爬取第8章：第七章 灵炉，意外
    页面标题：第七章 灵炉，意外, 内容长度：4355, 段落数：67
    当前章爬取完成
正在爬取第9章：第八章 骑士圣殿也来了
    页面标题：第八章 骑士圣殿也来了, 内容长度：3902, 段落数：66
    当前章爬取完成
正在爬取第10章：第九章 灵炉，再见灵炉
    页面标题：第九章 灵炉，再见灵炉, 内容长度：4261, 段落数：79
    当前章爬取完成
正在爬取第11章：第十章 绝望周末？
    页面标题：第十章 绝望周末？, 内容长度：4255, 段落数：90
    当前章爬取完成
正在爬取第12章：第十一章 骑士圣殿的邋遢骑士
    页面标题：第十一章 骑士圣殿的邋遢骑士, 内容长度：4011, 段落数：69
    当前章爬取完成
正在爬取第13章：第十二章 悲催的神圣骑士
    页面标题：第十二章 悲催的神圣骑士, 内容长度：3923, 段落数：67
    当前章爬取完成
正在爬取第14章：第十三章 胆小是优秀的素质
    页面标题：第十三章 胆小是优秀的素质, 内容长度：4154, 段落数：57
    当前章爬取完成
正在爬取第

In [14]:
txtfp.close()